#                                 DATA PREPOCESSING

### Imports 

In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import shutil
import os
import json
import csv

# WEATHER DATA

### Removing the datafiles of unwanted years from weather folder

In [3]:
weather_path = #path to the weather folder
os.chdir(weather_path)
airports = os.listdir()
for i in range(len(airports)):
    path = weather_path + airports[i]
    for fname in os.listdir(path):
        if fname.startswith("2013"):
            os.remove(os.path.join(path, fname))
            
    for fname in os.listdir(path):
        if fname.startswith("2014"):
            os.remove(os.path.join(path, fname))
            
    for fname in os.listdir(path):
        if fname.startswith("2015"):
            os.remove(os.path.join(path, fname))

### Converting the json files to csv 

In [4]:
os.chdir(weather_path)
for i in range(len(airports)):
    path1 = weather_path+airports[i]
    os.chdir(path1)
    reports = os.listdir()
    for j in range(len(reports)):
        if reports[j].endswith(".json"):
            path = path1 + '/' + reports[j]
            with open(path, 'r') as json_file:
                json_data  = json.load(json_file)
            weather_record = json_data['data']['weather']
            csv_data = open(os.path.splitext(path)[0]+'.csv', 'w')
            csv_writer = csv.writer(csv_data)
            count = 0
            for i in weather_record:
                if count == 0:
                    header = i.keys()
                    csv_writer.writerow(header)
                    count += 1
                csv_writer.writerow(i.values())
    csv_data.close()

#Seggregating the csv files
os.chdir(weather_path)
for i in range(len(airports)):
    path1 = weather_path+airports[i]
    os.mkdir(path1 + '/CSV FILES')

os.chdir(weather_path)
for i in range(len(airports)):
    path1 = weather_path+airports[i]
    os.chdir(path1)
    files = os.listdir()
    for i in range(len(files)):
        src = path1 + '/' + files[i]
        dest = path1 + '/CSV FILES'
        if files[i].endswith('.csv'):
            shutil.move(src, dest)

#### Viewing the weather data in csv format

In [5]:
os.chdir(weather_path+'ATL/CSV FILES/')
sample_weather = pd.read_csv('2016-1.csv')
print(sample_weather.shape)

(31, 10)


In [6]:
print(sample_weather.columns)

Index(['mintempC', 'maxtempF', 'sunHour', 'mintempF', 'maxtempC', 'hourly',
       'totalSnow_cm', 'date', 'astronomy', 'uvIndex'],
      dtype='object')


### Redefining weather data

In [7]:
os.chdir(weather_path)
for i in range(len(airports)):
    path1 = weather_path+airports[i]+'/CSV FILES'
    os.chdir(path1)
    files = os.listdir()
    for j in range(len(files)):
        df1 = pd.read_csv(path1 + '/' + files[j])
        
        df1 = df1.drop(['mintempC', 'maxtempF', 'sunHour', 'mintempF', 'maxtempC', 'totalSnow_cm', 'astronomy', 'uvIndex'], axis = 1)
        
        for k in range(len(df1['hourly'])):
            df1['hourly'][k] = json.loads(df1['hourly'][k].replace("'", '"'))
            
        unwanted = ['FeelsLikeF', 'FeelsLikeC', 'DewPointC', 'winddir16Point', 'windspeedMiles', 'HeatIndexF', 'HeatIndexC', 'weatherIconUrl', 'weatherDesc', 'tempC', 'WindChillC']
        for l in range(len(df1['hourly'])):
            for m in range(len(df1['hourly'][l])):
                #print(str2[i])
                for key in unwanted:
                    del df1['hourly'][l][m][key]
        
        df1 = df1.rename(columns = {'Unnamed: 0' : 'Airport'})
        
        df1['Airport'] = airports[i]
        
        df1.to_csv(path1 + '/' + files[j])
        


In [8]:
df1['hourly'][0][0]

{'windspeedKmph': '17',
 'DewPointF': '68',
 'cloudcover': '14',
 'precipMM': '0.0',
 'WindGustMiles': '17',
 'pressure': '1022',
 'WindGustKmph': '27',
 'visibility': '10',
 'weatherCode': '116',
 'tempF': '74',
 'WindChillF': '74',
 'winddirDegree': '94',
 'humidity': '82',
 'time': '0'}

In [9]:
for m in range(len(airports)):
    path1 = weather_path+airports[m]+'/CSV FILES'
    os.chdir(path1)
    files = os.listdir()
    for n in range(len(files)):
        data2 = pd.read_csv(files[n])
        for x in range(len(data2['hourly'])):
            data2['hourly'][x] = json.loads(data2['hourly'][x].replace("'", '"'))
        columns = data2.columns
        for v in range(len(columns)):
            if(columns[v] == "Unnamed: 0"):
                data2 = data2.drop(columns[v], axis = 1)
        
        data_new = pd.DataFrame(columns=['Airport', 'Date', 'Time', 'windspeedKmph', 'DewPointF', 'cloudcover', 'precipMM', 'pressure', 'WindGustKmph', 'visibility', 'weatherCode', 'tempF', 'WindChillF', 'winddirDegree', 'humidity'])

        for x in range(len(data2)):
            for y in range(len(data2['hourly'][x])):
                data_new = data_new.append({'Airport': data2['Airport'][x], 'Date': data2['date'][x], 'Time':data2['hourly'][x][y]['time'], 'windspeedKmph':data2['hourly'][x][y]['windspeedKmph'], 'DewPointF': data2['hourly'][x][y]['DewPointF'], 'cloudcover': data2['hourly'][x][y]['cloudcover'], 'precipMM': data2['hourly'][x][y]['precipMM'], 'pressure': data2['hourly'][x][y]['pressure'], 'WindGustKmph': data2['hourly'][x][y]['WindGustKmph'], 'visibility': data2['hourly'][x][y]['visibility'], 'weatherCode': data2['hourly'][x][y]['weatherCode'], 'tempF': data2['hourly'][x][y]['tempF'], 'WindChillF': data2['hourly'][x][y]['WindChillF'], 'winddirDegree': data2['hourly'][x][y]['winddirDegree'], 'humidity': data2['hourly'][x][y]['humidity']}, ignore_index=True)
        
        data_new.to_csv(files[n])

/Users/sowmiya/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
os.chdir(weather_path+'ATL/CSV FILES/')
df = pd.read_csv('2016-1.csv')
df = df.drop("Unnamed: 0", axis=1)
df.columns

Index(['Airport', 'Date', 'Time', 'windspeedKmph', 'DewPointF', 'cloudcover',
       'precipMM', 'pressure', 'WindGustKmph', 'visibility', 'weatherCode',
       'tempF', 'WindChillF', 'winddirDegree', 'humidity'],
      dtype='object')

## FLIGHT DATA

### Moving the flight csv files directly into flight_data folder

In [11]:
flight_data_2016 = []
flight_data_2017 = []
flight_path = #path to the folder containing the flight details
os.chdir(flight_path)
for fname in os.listdir(flight_path):
        if fname.endswith(".DS_Store"):
            os.remove(os.path.join(flight_path, fname))
            
year_2016_path = #2016 subfolder in flight folder
year_2017_path = #2017 subfolder in flight folder

months_2016 = os.listdir(year_2016_path)
for i in range(len(months_2016)):
    files_2016 = year_2016_path + months_2016[i]
    list1 = os.listdir(files_2016)
    for j in list1:
        if j.endswith(".csv"):
            flight_data_2016.append(j)            
months_2017 = os.listdir(year_2017_path)
for k in range(len(months_2017)):
    files_2017 = year_2017_path + months_2017[k]
    list2 = os.listdir(files_2017)
    for l in list2:
        if l.endswith(".csv"):
            flight_data_2017.append(l)

for x in range(len(flight_data_2016)):
    src = year_2016_path + months_2016[x] +'/' + flight_data_2016[x]
    dest = flight_path
    shutil.move(src, dest)
    
for y in range(len(flight_data_2017)):
    src = year_2017_path + months_2017[y] +'/' + flight_data_2017[y]
    dest = flight_path
    shutil.move(src, dest)

### Redefining flight data

In [12]:
os.chdir(flight_path)

#creating a list of flight csv files
for data in os.listdir(flight_path):
        if data.endswith(".DS_Store"):
            os.remove(os.path.join(flight_path, data))
flight_data = os.listdir(flight_path)

#unnecassary columns to drop
drop = ['DayOfWeek', 'UniqueCarrier', 'AirlineID', 'Carrier', 'TailNum', 'FlightNum', 'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDelay', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'LateAircraftDelay', 'TaxiIn', 'ArrDelay', 'ArrivalDelayGroups', 'ArrTimeBlk', 'Cancelled', 'CancellationCode', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Flights', 'Distance', 'DistanceGroup', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'FirstDepTime', 'TotalAddGTime', 'LongestAddGTime', 'DivAirportLandings', 'DivReachedDest', 'DivActualElapsedTime', 'DivArrDelay', 'DivDistance', 'Div1Airport', 'Div1AirportID', 'Div1AirportSeqID', 'Div1WheelsOn', 'Div1TotalGTime', 'Div1LongestGTime', 'Div1WheelsOff', 'Div1TailNum', 'Div2Airport', 'Div2AirportID', 'Div2AirportSeqID', 'Div2WheelsOn', 'Div2TotalGTime', 'Div2LongestGTime', 'Div2WheelsOff', 'Div2TailNum', 'Div3Airport', 'Div3AirportID', 'Div3AirportSeqID', 'Div3WheelsOn', 'Div3TotalGTime', 'Div3LongestGTime', 'Div3WheelsOff', 'Div3TailNum', 'Div4Airport', 'Div4AirportID', 'Div4AirportSeqID', 'Div4WheelsOn', 'Div4TotalGTime', 'Div4LongestGTime', 'Div4WheelsOff', 'Div4TailNum', 'Div5Airport', 'Div5AirportID', 'Div5AirportSeqID', 'Div5WheelsOn', 'Div5TotalGTime', 'Div5LongestGTime', 'Div5WheelsOff', 'Div5TailNum', 'Unnamed: 109']

#since each of the csv file is huge, we read the csv through dask and then drop the columns in drop[] list
for i in range(len(flight_data)):
    solved_data = dd.read_csv(flight_data[i], dtype = {'Div2Airport': 'object', 'Div2TailNum':'object', 'ArrTime': 'float64','CancellationCode': 'object','DepTime': 'float64','DepartureDelayGroups': 'float64','WheelsOff': 'float64','WheelsOn': 'float64', 'Div1TailNum': 'object', 'DivAirportLandings': 'float64', 'Div3Airport': 'object', 'Div1Airport': 'object', 'ArrivalDelayGroups': 'float64'})
    solved_data = solved_data.compute()
    for j in range(len(drop)):
        solved_data = solved_data.drop(drop[j], axis = 1)
    solved_data.to_csv(flight_data[i])

#selecting only those rows that have station codes for which the weather data is provided(both origin and destination)
for j in (flight_data):
    d1 = pd.read_csv(j)
    columns1 = d1.columns
    for m in range(len(columns1)):
        if(columns1[m] == "Unnamed: 0"):
            d1 = d1.drop(columns1[m], axis = 1)
            
    d2 = d1[(d1['Origin'].isin(airports))]
    d3 = d2[d2['Dest'].isin(airports)]
    d3['CRSArrTime_round'] = ""
    
    #rounding off the CRSArrTime to merge it with the weather data of the Destination station
    index = d3.index
    def truncate(n):
        return n%100
    for y in index:
        dec = truncate(d3['CRSArrTime'][y])
        if(dec <= 30):
            d3['CRSArrTime_round'][y] = int(d3['CRSArrTime'][y]/100) * 100
        elif(dec > 30):
            d3['CRSArrTime_round'][y] = (int(d3['CRSArrTime'][y]/100)+1) * 100
        if(d3['CRSArrTime_round'][y] > 2300):
            d3['CRSArrTime_round'][y] = 0
    d3.to_csv(j)

/Users/sowmiya/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/sowmiya/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/sowmiya/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [13]:
os.chdir('/Users/Sowmiya/SF/MLProject/Sample/Flight Data/')
df = pd.read_csv('On_Time_On_Time_Performance_2016_1.csv')
df = df.drop("Unnamed: 0", axis=1)
#df.head(2)
print(df.columns)
print(df.shape)

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'FlightDate',
       'OriginAirportID', 'Origin', 'DestAirportID', 'Dest', 'CRSDepTime',
       'DepTime', 'DepDelayMinutes', 'DepDel15', 'CRSArrTime', 'ArrTime',
       'ArrDelayMinutes', 'ArrDel15', 'CarrierDelay', 'CRSArrTime_round'],
      dtype='object')
(75010, 19)


# COMBINING THE FLIGHT AND WEATHER DATA

### combining all the flight data files into one

In [14]:
combined = pd.DataFrame(columns = ['Year', 'Quarter', 'Month', 'DayofMonth', 'FlightDate', 'OriginAirportID', 'Origin', 'DestAirportID', 'Dest', 'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDel15', 'CRSArrTime', 'ArrTime', 'ArrDelayMinutes', 'ArrDel15', 'CRSArrTime_round','CarrierDelay'])
for x in flight_data:
    os.chdir(flight_path)
    df = pd.read_csv(x)
    
    columns = df.columns
    for m in range(len(columns)):
        if(columns[m] == "Unnamed: 0"):
            df = df.drop(columns[m], axis = 1)
    
    combined = pd.concat([combined, df])
    
    os.chdir(#path to the data folder)
    combined.to_csv('flight.csv')

### combining all weather data files into one

In [15]:
for i in airports:
    airport_combined = pd.DataFrame(columns = ['Airport', 'Date', 'Time', 'windspeedKmph', 'DewPointF',
       'cloudcover', 'precipMM', 'pressure', 'WindGustKmph', 'visibility',
       'weatherCode', 'tempF', 'WindChillF', 'winddirDegree', 'humidity'])
    os.chdir(weather_path + i + '/CSV FILES/')
    files = os.listdir()
    for j in range(len(files)):
        os.chdir(weather_path + i + '/CSV FILES/')
        df = pd.read_csv(files[j])
        columns = df.columns
        for m in range(len(columns)):
            if(columns[m] == "Unnamed: 0"):
                df = df.drop(columns[m], axis = 1)
            
        airport_combined = pd.concat([airport_combined, df])
    
        os.chdir(weather_path)
        airport_combined.to_csv(i + '.csv')
        
        
os.chdir(weather_path)
stations = []
for x in os.listdir():
    if(x.endswith('.csv')):
        stations.append(x)
combined =  pd.DataFrame(columns = ['Airport', 'Date', 'Time', 'windspeedKmph', 'DewPointF',
       'cloudcover', 'precipMM', 'pressure', 'WindGustKmph', 'visibility',
       'weatherCode', 'tempF', 'WindChillF', 'winddirDegree', 'humidity'])
for y in stations:
    os.chdir(weather_path)
    df1 = pd.read_csv(y)
    
    columns = df1.columns
    for m in range(len(columns)):
        if(columns[m] == "Unnamed: 0"):
            df1 = df1.drop(columns[m], axis = 1)
            
    combined = pd.concat([combined, df1])
    
    os.chdir(#path to the data folder')
    combined.to_csv('weather.csv')

### complete merge of flight and weather data into single file

In [16]:
os.chdir(#path to data folder)

df1 = pd.read_csv('flight.csv')
columns1 = df1.columns
for m in range(len(columns1)):
    if(columns1[m] == "Unnamed: 0"):
        df1 = df1.drop(columns1[m], axis = 1)

df2 = pd.read_csv('weather.csv')
columns2 = df1.columns
for m in range(len(columns2)):
    if(columns2[m] == "Unnamed: 0"):
        df1 = df1.drop(columns2[m], axis = 1)

df3 = pd.merge(df1, df2, left_on = ['FlightDate', 'Dest', 'CRSArrTime_round'], right_on = ['Date', 'Airport', 'Time'])

df3.to_csv('data.csv')


#  REDEFINING THE FINAL MERGED DATAFRAME

In [17]:
os.chdir(#path to data folder)
x = pd.read_csv('data.csv')
x = x.drop("Unnamed: 0", axis=1)
x

,Year,Quarter,Month,DayofMonth,FlightDate,OriginAirportID,Origin,DestAirportID,Dest,CRSDepTime,...,cloudcover,precipMM,pressure,WindGustKmph,visibility,weatherCode,tempF,WindChillF,winddirDegree,humidity
0,2016,3,8,27,2016-08-27,11292,DEN,12889,LAS,600,...,47,0.0,1012,8,10,113,78,77,30,49
1,2016,3,8,27,2016-08-27,12892,LAX,12889,LAS,620,...,47,0.0,1012,8,10,113,78,77,30,49
2,2016,3,8,27,2016-08-27,12889,LAS,13204,MCO,736,...,60,0.7,1016,23,10,386,84,85,84,53
3,2016,3,8,27,2016-08-27,10397,ATL,13204,MCO,1350,...,60,0.7,1016,23,10,386,84,85,84,53
4,2016,3,8,27,2016-08-27,11298,DFW,13204,MCO,1120,...,60,0.7,1016,23,10,386,84,85,84,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877291,2016,2,5,11,2016-05-11,11292,DEN,11057,CLT,2345,...,2,0.0,1018,18,10,113,69,68,192,75
1877292,2016,2,5,11,2016-05-11,11292,DEN,10397,ATL,2359,...,0,0.0,1019,14,10,113,69,69,219,84
1877293,2016,2,5,11,2016-05-11,12889,LAS,13303,MIA,2300,...,4,0.0,1020,19,10,113,75,75,124,82
1877294,2016,2,5,12,2016-05-12,12889,LAS,10397,ATL,2130,...,1,0.0,1019,6,10,113,68,70,236,71


### check for null values and if any occurs, dropping them

In [18]:
x.isnull().sum()

Year                      0
Quarter                   0
Month                     0
DayofMonth                0
FlightDate                0
OriginAirportID           0
Origin                    0
DestAirportID             0
Dest                      0
CRSDepTime                0
DepTime               20980
DepDelayMinutes       20987
DepDel15              20987
CRSArrTime                0
ArrTime               22080
ArrDelayMinutes       25860
ArrDel15              25860
CRSArrTime_round          0
CarrierDelay        1489238
Unnamed: 0.1              0
Airport                   0
Date                      0
Time                      0
windspeedKmph             0
DewPointF                 0
cloudcover                0
precipMM                  0
pressure                  0
WindGustKmph              0
visibility                0
weatherCode               0
tempF                     0
WindChillF                0
winddirDegree             0
humidity                  0
dtype: int64

In [19]:
x = x.fillna(0)

In [20]:
x.isnull().sum()

Year                0
Quarter             0
Month               0
DayofMonth          0
FlightDate          0
OriginAirportID     0
Origin              0
DestAirportID       0
Dest                0
CRSDepTime          0
DepTime             0
DepDelayMinutes     0
DepDel15            0
CRSArrTime          0
ArrTime             0
ArrDelayMinutes     0
ArrDel15            0
CRSArrTime_round    0
CarrierDelay        0
Unnamed: 0.1        0
Airport             0
Date                0
Time                0
windspeedKmph       0
DewPointF           0
cloudcover          0
precipMM            0
pressure            0
WindGustKmph        0
visibility          0
weatherCode         0
tempF               0
WindChillF          0
winddirDegree       0
humidity            0
dtype: int64

### Removing unnecessary columns 

In [21]:
x = x.drop(['Unnamed: 0.1', 'Airport', 'Date', 'Time'], axis = 1)

In [22]:
x.shape

(1877296, 31)

### Finally writing the changes back to the file

In [23]:
x.to_csv('data.csv')

# DONE